In [1]:
import cv2
import numpy as np

In [2]:
# Tải mô hình đã được huấn luyện trước:
net = cv2.dnn.readNetFromCaffe("./moudules/deploy.prototxt",
                               "./moudules/res10_300x300_ssd_iter_140000_fp16.caffemodel")

# Mở camera:
cam = cv2.VideoCapture(0)


while True:
    # Đọc khung hình
    ret, frame = cam.read()
    if(ret != True):
        break
    
    # Chuẩn bị dữ liệu đầu vào:
        # img: là ảnh muốn nhận diện khuôn mặt
        # 1.0: Đây là tỉ lệ co dãn cho hình ảnh
        # (300,300): kích thước ảnh yêu cầu cho đầu vào
        # (104,177,123): màu sắc trung bình
        # swapRB: không hoán đổi màu đỏ và xanh
    blob = cv2.dnn.blobFromImage(frame, 1.0, (300,300), (104,177,123), swapRB= False)

    # Dặt dữ liệu đầu vào cho ảnh:
    net.setInput(blob)

    # Chạy qua từng pixel trong mảng để phát hiện khuôn mặt:
    faces = net.forward()

    # Lấy kích thước của ảnh đầu vào:
    h = frame.shape[0]
    w = frame.shape[1]

    # Duyệt từng khuôn mặt được phát hiện:
    for i in range(0, faces.shape[2]):
        confidence = faces[0,0,i,2]
    
        # kiểm tra nếu độ tin cậy hơn 0.5:
        if(confidence>0.5):
            # Trích xuất toạ độ
            startx = int(faces[0,0,i,3]*w)
            starty = int(faces[0,0,i,4]*h)
            endx = int(faces[0,0,i,5]*w)
            endy = int(faces[0,0,i,6]*h)

            # Vẽ hcn xung quanh mặt
            cv2.rectangle(frame, (startx,starty), (endx,endy), (0,255,0), 2)

            # hiển thị độ tin cây:
            text = "Face: {:.2f}%".format(confidence*100)
            cv2.putText(frame, text, (startx,starty-10), cv2.FONT_HERSHEY_SCRIPT_SIMPLEX,1,(255,255,255))

    # Hiển thị ảnh gốc:
    cv2.imshow("Result",frame)
    if(cv2.waitKey(1)==ord("q")):
        break

cam.release()
cv2.destroyAllWindows()